<a href="https://colab.research.google.com/github/soufianefiqi/John-Travolta/blob/main/B21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder


In [2]:
data = pd.read_csv('B21.csv',sep=';')
data

FileNotFoundError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
data.isnull().sum()

It seems to be a lot of missing data, let's try to clear them

In [ ]:
data_clear=data.dropna()
data_clear


We only left with 68 rows of data (which is obviously not good). Let's try another way around

First, we find the least correlated variable with the "status banjir"

In [ ]:
data_2=data

Kemayoran_ddd_car_dummies = pd.get_dummies(data_2.Kemayoran_ddd_car, prefix='Kemayoran_ddd_car').iloc[:, 1:]

data_2 = pd.concat([data_2, Kemayoran_ddd_car_dummies], axis=1)

TJP_ddd_car_dummies = pd.get_dummies(data_2.TJP_ddd_car, prefix='TJP_ddd_car').iloc[:, 1:]

data_2 = pd.concat([data_2, TJP_ddd_car_dummies], axis=1)

Bogor_ddd_car_dummies = pd.get_dummies(data_2.Bogor_ddd_car, prefix='Bogor_ddd_car').iloc[:, 1:]

data_2 = pd.concat([data_2, Bogor_ddd_car_dummies], axis=1)

del data_2['Kemayoran_ddd_car']
del data_2['TJP_ddd_car']
del data_2['Bogor_ddd_car']

data_2.head()

In [ ]:
corre = data_2.corr(method ='pearson') 
corre.head()

In [ ]:
corre.to_csv("correlation.csv")

lets do some analysis in excel

after analysis, we can see that the variables that has the correlation lower than 0.05 are:
 

---


TJP_ddd_car_W ;
TJP_ddd_car_SE;
Bogor_ddd_car_NE;
Kemayoran_ss;
Bogor_ff_avg;
TJP_Tn;
TJP_RH_avg;
Kemayoran_ff_x;
Kemayoran_ddd_car_SW;
TJP_ddd_car_S ;
Bogor_ddd_car_E; 
Bogor_ddd_car_SE;
TJP_ff_avg;
Kemayoran_ddd_x;
TJP_ddd_car_N;


after analysis, we can also see that the variables that has the correlation lower than 0.1(greater than 0.05) are:



---

TJP_ss; TJP_ff_x; Bogor_ddd_x; Bogor_ddd_car_S; TJP_ddd_car_NE; TJP_ddd_car_E; Kemayoran_ff_avg; Kemayoran_RR; Bogor_ddd_car_NW



after analysis, we can also see that 10 variables that has the highest correlation are



---

Kemayoran_Tx;
Bogor_Tx;
Kemayoran_ddd_car_N ;
Bogor_Tavg;
Kemayoran_Tavg;
Bogor_RH_avg;
TJP_Tx;
Bogor_ddd_car_N ;
Bogor_ss;
Kemayoran_ddd_car_NW;

lets take all 10 variables into account

# Model Building

In [ ]:
needed_cols = ['Kemayoran_Tx', 'Bogor_Tx', 'Kemayoran_ddd_car_N ', 'Bogor_Tavg', 'Kemayoran_Tavg', 'Bogor_RH_avg', 'TJP_Tx', 'Bogor_ddd_car_N ', 'Bogor_ss', 'Kemayoran_ddd_car_NW','status_banjir']

In [ ]:
data_3=data_2[needed_cols]
data_3.isnull().sum()

Then we do some data clearing

In [ ]:
data_4=data_3.dropna()

In [ ]:
print(data_4.loc[data_4['status_banjir'] == 1].shape)
print(data_4.loc[data_4['status_banjir'] == 0].shape)

yup we still has enough data, we can keep going

In [ ]:
feature_cols = ['Kemayoran_Tx', 'Bogor_Tx', 'Kemayoran_ddd_car_N ', 'Bogor_Tavg', 'Kemayoran_Tavg', 'Bogor_RH_avg', 'TJP_Tx', 'Bogor_ddd_car_N ', 'Bogor_ss', 'Kemayoran_ddd_car_NW']

In [ ]:
X = data_4[feature_cols]
y = data_4.status_banjir

In [ ]:
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

In [ ]:
from sklearn.tree import export_graphviz
with open("A", 'w') as f:
    f = export_graphviz(treeclf, out_file=f, feature_names=feature_cols)

In [ ]:
import graphviz
from IPython.display import display
display(graphviz.Source(export_graphviz(treeclf)))

# Model Checking

In [ ]:
pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
treeclf2 = DecisionTreeClassifier()

treeclf2 = treeclf2.fit(X_train,y_train)

y_pred = treeclf2.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Building (Take 2)

We only take accounts the column that has importance greater than 0.05 in the first model

In [ ]:
feature_cols = ['Kemayoran_Tx', 'Kemayoran_ddd_car_N ', 'Bogor_Tavg', 'Kemayoran_Tavg', 'Bogor_RH_avg', 'TJP_Tx']
X = data_4[feature_cols]
y = data_4.status_banjir

from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

In [ ]:
from sklearn.tree import export_graphviz
with open("A", 'w') as f:
    f = export_graphviz(treeclf, out_file=f, feature_names=feature_cols)

import graphviz
from IPython.display import display
display(graphviz.Source(export_graphviz(treeclf)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
treeclf2 = DecisionTreeClassifier()

treeclf2 = treeclf2.fit(X_train,y_train)

y_pred = treeclf2.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
treeclf2 = DecisionTreeClassifier()

treeclf2 = treeclf2.fit(X_train,y_train)

y_pred = treeclf2.predict(X_test)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})